In [1]:
%reset -fs

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.decomposition import TruncatedSVD, NMF

import re



In [2]:
df = pd.read_csv('eu_tweet_df_eda1.csv')

/var/folders/gc/kgfpjt4x1d5c1thdk2sl_2hr0000gn/T/ipykernel_16054/3349357611.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('eu_tweet_df_eda1.csv')


In [3]:
def strip_punc_lower_http(txt):
    txt = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", txt)

    txt = re.sub(pattern= "[^\w\s]",
            repl="",
            string=txt).lower()
    
    txt = re.sub(pattern= "[0-9]",
            repl="",
            string=txt)
    
    return txt
strip_punc_lower_http('I am, testing #ALEX > than X 10. https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html')

'i am testing alex  than x   '

In [4]:
df['text'] = df['text'].apply(lambda x: strip_punc_lower_http(x))

In [5]:
df.head()

,id,created_at,text,user_name,user_screen_name,user_id,user_description,user_location,user_verified,lang,...,in_reply_to_user_id,retweet_count,retweet_id,retweet_screen_name,user_created_at,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count
0,866239077045207040,Sun May 21 10:27:44 +0000 2017,pounding the streets of redland this morning w...,Green💚🌻Molly,GreenPartyMolly,726372601,Molly Scott Cato 👋 Green Party External Comms ...,South West,False,en,...,NaN,23.0,8.662375e+17,bristol_yg,Mon Jul 30 16:21:34 +0000 2012,27154.0,59652.0,1965.0,854.0,69671.0
1,866232717196840960,Sun May 21 10:02:28 +0000 2017,the ukip battle bus is coming please support m...,Dr Julia Reid PhD,julia_reid,43554444,Former Brexit Party MEP for the SW (2014-19) -...,Calne,True,en,...,NaN,38.0,8.662321e+17,DrTeckKhong,Sat May 30 16:46:20 +0000 2009,62208.0,4773.0,6282.0,242.0,139670.0
2,866236815463862272,Sun May 21 10:18:45 +0000 2017,yesterday was th european maritime day today i...,Catherine Bearder🇪🇺 🇺🇦🇺🇦🇪🇺🔶,catherinemep,454779231,Lib Dem. Board of IFAW and Council of Unlock D...,Oxford,True,en,...,NaN,21.0,8.662204e+17,EU_MARE,Wed Jan 04 11:13:08 +0000 2012,2705.0,32987.0,9121.0,677.0,36328.0
3,866236868404314112,Sun May 21 10:18:57 +0000 2017,thesnp depute leader and candidate for moray a...,Ian Hudghton,HudghtonSNP,308677137,SNP President 2005 - 2020. SNP Member of the E...,Scotland,False,en,...,NaN,6.0,8.662227e+17,erikgeddes,Tue May 31 20:22:38 +0000 2011,21.0,5044.0,398.0,205.0,1789.0
4,866233710714257408,Sun May 21 10:06:25 +0000 2017,cacophonic that and fox hunting and dementia t...,Catherine Bearder🇪🇺 🇺🇦🇺🇦🇪🇺🔶,catherinemep,454779231,Lib Dem. Board of IFAW and Council of Unlock D...,Oxford,True,en,...,25067514.0,0.0,NaN,NaN,Wed Jan 04 11:13:08 +0000 2012,2705.0,32987.0,9121.0,677.0,36328.0


In [6]:
df_0_500k = df.loc[:500000,['id', 'created_at', 'text']].copy()

In [7]:
df_0_500k

,id,created_at,text
0,866239077045207040,Sun May 21 10:27:44 +0000 2017,pounding the streets of redland this morning w...
1,866232717196840960,Sun May 21 10:02:28 +0000 2017,the ukip battle bus is coming please support m...
2,866236815463862272,Sun May 21 10:18:45 +0000 2017,yesterday was th european maritime day today i...
3,866236868404314112,Sun May 21 10:18:57 +0000 2017,thesnp depute leader and candidate for moray a...
4,866233710714257408,Sun May 21 10:06:25 +0000 2017,cacophonic that and fox hunting and dementia t...
...,...,...,...
499996,959491023046049792,Fri Feb 02 18:17:41 +0000 2018,united states congressman rephuffman took time...
499997,959490706606829568,Fri Feb 02 18:16:25 +0000 2018,were lifting the public sector pay cap mean...
499998,959491686048071680,Fri Feb 02 18:20:19 +0000 2018,a winter scene in the yorkshiredales this morn...
499999,959490755168555008,Fri Feb 02 18:16:37 +0000 2018,russia interfered in the election\nrussia wil...


In [8]:
my_stop_words = ['wonderful', 'did', 'didnt', 'issue', 'issues', 'actually', 'id','th',
                'community']

In [9]:
my_stop_words = text.ENGLISH_STOP_WORDS.union(my_stop_words)

In [10]:
tv_30docs = TfidfVectorizer(stop_words = my_stop_words, min_df=30)
dtm_tv_30docs = tv_30docs.fit_transform(list(df_0_500k['text']))

In [11]:
dtm_tv_30docs.shape

(500001, 16604)

In [37]:
nmf_topics = NMF(n_components=50, init='random', max_iter=1000)
nmf_17k_words = nmf_topics.fit_transform(dtm_tv_30docs)

In [34]:
# Function to display the top n terms in each topic
def display_topics(model, feature_names, no_top_words, topic_names = None): 
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix + 1)
        else:
            print("\nTopic: ", topic_names[ix])
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print("\n")
    return model, feature_names, no_top_words

In [38]:
output_17kwords = display_topics(nmf_topics, tv_30docs.get_feature_names_out(), 10,
                                          topic_names=None)


Topic  1
amp, jobs, security, govt, communities, future, economy, st, protect, food

Topic  2
im, sure, proud, sorry, going, afraid, ive, glad, working, pleased

Topic  3
morning, live, join, tomorrow, come, ill, watch, discuss, tune, talk

Topic  4
dont, think, forget, miss, agree, believe, understand, worry, really, tell

Topic  5
eu, citizens, rights, trade, leave, deal, member, countries, europe, withdrawal

Topic  6
happy, birthday, wishing, christmas, friend, rep, celebrating, wish, colleague, friends

Topic  7
support, campaign, proud, count, educator, delighted, strong, continued, pleased, appreciate

Topic  8
brexit, deal, hard, negotiations, impact, britain, ireland, talks, british, tory

Topic  9
public, important, school, funding, students, count, educator, sector, pay, services

Topic  10
good, luck, meet, news, hear, point, results, really, morning, idea

Topic  11
news, families, thoughts, victims, prayers, family, attack, hear, affected, friends

Topic  12
thank, kind,

In [39]:
total = 0
for i in range(50):
    print(f'topic{i+1} {sum(nmf_topics.components_[i])}')
    total += sum(nmf_topics.components_[i])
total

topic1 10.363245253097773
topic2 5.24242063463786
topic3 11.585186954817017
topic4 5.012926783111181
topic5 11.859527756796156
topic6 4.606922238661615
topic7 7.326534370755414
topic8 12.714842071345219
topic9 8.679372862563376
topic10 6.228686613393258
topic11 15.670551152707318
topic12 4.782228958498796
topic13 9.044579285853903
topic14 18.61141235323843
topic15 14.328326884866211
topic16 6.052785463749857
topic17 11.641713782595257
topic18 8.635326567662325
topic19 11.49381703726601
topic20 10.611886795197753
topic21 15.53577644969546
topic22 16.496345404194617
topic23 10.627927021202316
topic24 13.687153898837394
topic25 16.188019370571933
topic26 10.45596604830446
topic27 11.01152435193205
topic28 14.859120572790983
topic29 15.450413594233254
topic30 12.949285754227452
topic31 14.747062529847993
topic32 7.9828159120939155
topic33 10.858833518543491
topic34 8.980171322807916
topic35 16.95961998390954
topic36 10.431477739688399
topic37 23.231286261741456
topic38 19.7380419414569
top

654.3443536949575